# Topics API: Model Classifier
This colab makes it easy to load the TensorFlow Lite model used by Chrome to infer topics from hostnames.

The classifier is stored `model.tflite`. The file `override_list.pd` contains the topics of 50k hostnames manually annotated by Google.



# Libraries

In [2]:
%pip install grpcio

Defaulting to user installation because normal site-packages is not writeable
  Using cached grpcio-1.62.2.tar.gz (26.3 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [852 lines of output]
      Found cython-generated files...
      running bdist_wheel
      running build
      running build_py
      running build_project_metadata
      creating pyb
      creating pyb/lib.macosx-10.15-x86_64-cpython-310
      creating pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_typing.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_compression.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_channel.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_common.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/pytho

In [3]:
%pip install tflite-support-nightly protobuf-compiler


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 MB 2.6 MB/s eta 0:00:0000:0100:01
  Using cached protobuf_compiler-1.0.20-py3-none-any.whl (8.6 kB)
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Using cached sounddevice-0.4.6-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached tqdm-4.31.1-py2.py3-none-any.whl (48 kB)
  Using cached grpcio-tools-1.18.0.tar.gz (1.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached bleach-2.1-py2.py3-none-any.whl (27 kB)
  Using cached grpcio-

In [20]:
proto_path = 'page_topics_override_list.proto'

example_proto = b"""
syntax = "proto2";

// The whole override list.
message PageTopicsOverrideList {
  repeated PageTopicsOverrideEntry entries = 1;
}

// An individual entry in the override list. |domain| is expected to be the
// exact string input that is otherwise passed to the TFLite model, with all
// needed pre-processing and/or cleaning done to it already.
message PageTopicsOverrideEntry {
  optional string domain = 1;
  optional AnnotatedPageTopics topics = 2;
}

// The topic identifiers for the given domain.
message AnnotatedPageTopics {
  repeated int32 topic_ids = 1;
}
"""

with open(proto_path, 'wb') as f:
  f.write(example_proto)

!cd /tmp/
!protoc "page_topics_override_list.proto" --python_out=.
%cd /Users/abirbenzaamia/Downloads/testing_privacy_sandbox/TopicsClassification
!rm "override_list.pb"
!gzip -dk "override_list.pb.gz"

/Users/abirbenzaamia/Downloads/testing_privacy_sandbox/TopicsClassification


In [19]:
%cd /Users/abirbenzaamia/Downloads/testing_privacy_sandbox/TopicsClassification

/Users/abirbenzaamia/Downloads/testing_privacy_sandbox/TopicsClassification


/Users/abirbenzaamia/Library/Python/3.10/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [21]:
import page_topics_override_list_pb2

# Read the existing address book.
override_list = page_topics_override_list_pb2.PageTopicsOverrideList()
try:
  f = open("override_list.pb", "rb")
  override_list.ParseFromString(f.read())
  f.close()
except IOError:
  print("Could not open file.")

# Replaces a set of common domain characters with white space. See https://source.chromium.org/chromium/chromium/src/+/main:components/optimization_guide/core/page_topics_model_executor.cc;l=211?q=meaningless%20f:optimization_guide&ss=chromium
def process_domain(domain):
  replace_chars = ['-', '_', '.', '+']
  for rc in replace_chars:
    domain = domain.replace(rc, " ")
  return domain

def check_override_list(override_list, domain):
  if override_list is None:
    return None
  for entry in override_list.entries:
    if entry.domain == domain:
      return entry.topics.topic_ids
  return None


# Override List

# Taxonomy

In [38]:
import json

def read_json_to_map(filename):
    map_list={}
    with open(filename, 'r') as file:
        data_map = json.load(file)
    return {int(key): value for key, value in data_map.items()} 

# Example usage:
cat_map = read_json_to_map('Data/taxonomy_v2.json')
#print(cat_map)

# Model Execution


In [42]:
%pip install tflite_support

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 MB 20.3 MB/s eta 0:00:0000:0100:01
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached sounddevice-0.4.6-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
  Using cached cffi-1.16.0-cp310-cp310-macosx_10_9_x86_64.whl (182 kB)

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:
import tflite_support.task
from tflite_support.task import text
from tflite_support.task import core

# Remember, this will fail if you haven't uploaded model.tflite. 
# See instructions at the top of this file.

options = text.BertNLClassifierOptions(
    base_options=core.BaseOptions(
        file_name='model.tflite'))

tflite_topics = text.BertNLClassifier.create_from_options(options)

def CategorySort(elem):
  return elem.score

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [44]:
domains = [
           "github.com",
           "wikipedia.org",
           "wikipedia.com", 
]

In [45]:
for domain in domains:
  print("domain: ", domain)
  processed_domain = process_domain(domain)
  print(processed_domain)
  topics = check_override_list(override_list, processed_domain)
  if topics != None:
    print(topics)
    for c in topics:
      print("Category: {} \t".format( cat_map[c]))
  else:
    topics = tflite_topics.classify(processed_domain)
    cats = sorted(topics.classifications[0].categories, key=CategorySort)[-5:][::-1]
    for c in cats:
      print("Category: {} \t - Score: {}".format( cat_map[int(c.category_name)],  c.score))
    
  print("\n")

domain:  github.com
github com
[534, 140, 536, 126]
Category: Internet & Telecom/Web Services/Cloud Storage 	
Category: Computers & Electronics/Software 	
Category: Internet & Telecom/Web Services/Web Hosting & Domain Registration 	
Category: Computers & Electronics 	


domain:  wikipedia.org
wikipedia org
[]


domain:  wikipedia.com
wikipedia com
Category: Jobs & Education/Education 	 - Score: 0.9149566292762756
Category: Computers & Electronics/Software 	 - Score: 0.07737835496664047
Category: Jobs & Education/Jobs/Career Resources & Planning 	 - Score: 0.019042151048779488
Category: Jobs & Education/Jobs/Job Listings 	 - Score: 0.010259849019348621
Category: Jobs & Education/Education/Distance Learning 	 - Score: 0.009897008538246155


